# This notebook is designed for teaching/testing purposes to help you visualize the tensor shapes that go through each module. Read it side-by-side with 'model.py'

In [1]:
# my virtual environments are rarely properly connected to jupyter so this fixes that. 
# you won't need this cell but running it won't hurt anything either
import sys
import os
current_dir = os.getcwd()  # Get the current working directory
venv_dir = os.path.join(current_dir, '../venv') 
python_version = str(sys.version_info.major) + '.' + str(sys.version_info.minor)
site_packages_path = os.path.join(venv_dir, 'lib', 'python' + python_version, 'site-packages')
sys.path.append(site_packages_path) 

In [2]:
# tokenizer
sys.path.append("..")  # Adds the parent directory to the path so we can see the tokenizer
from tokenizer_TinyStories import *
size = 512 # size options are 128, 256, 512 and 1024
path = f'../tokenizers/tiny_stories_tokenizer_{size}.model'
tokenizer = get_tokenizer(path) 

# config file
from config import *
cfg = Config()
cfg.vocab_len = tokenizer.vocab_len
print(cfg)

# model modules
from model import *

# Norms

In [4]:
### RMSNorm

# Create an instance of RMSNorm
hold = cfg.norm_type
cfg.norm_type = 'rmsnorm' # purposely mis-typing it
module = Norm(cfg.dim, cfg)

# let's take a look
print(sum(p.numel() for p in module.parameters())/1e3, 'K parameters')
print(module)

# Initially, logging is disabled by default
module.enable_logging()

### Optionally disabling printing for sub-functions
#module.disable_function_logging('CosineNorm')
#module.disable_function_logging('LayerNorm')
#module.disable_function_logging('RMSNorm')

x = torch.randn(32,cfg.max_seq_len,cfg.dim)

# Call the forward method - logging will occur
output = module(x)

# Disable logging. 
# This isn't actually necessary since we won't be using this object again but that's how you'd do it
module.disable_logging()

# clearing up ram jic we're training later
cfg.norm_type = hold
del hold, module, x, output

norm type rmsnorm not found. defaulting to RMSNorm
0.256 K parameters
Norm()

==========Entering Norm.forward==========
Inputs:
Tensor 'x' shape: torch.Size([32, 512, 128])

==========Entering Norm.RMSNorm==========
Inputs:
Tensor 'x' shape: torch.Size([32, 512, 128])

Outputs:
Tensor 'output' shape: torch.Size([32, 512, 128])
==========Exiting Norm.RMSNorm==========

Outputs:
Tensor 'output' shape: torch.Size([32, 512, 128])
==========Exiting Norm.forward==========


In [5]:
# LayerNorm
hold = cfg.norm_type
cfg.norm_type = 'LayerNorm'
module = Norm(cfg.dim, cfg)
module.enable_logging()

### Optionally disabling printing for sub-functions
#module.disable_function_logging('CosineNorm')
#module.disable_function_logging('LayerNorm')
#module.disable_function_logging('RMSNorm')

x = torch.randn(32,cfg.max_seq_len,cfg.dim)
output = module(x)
module.disable_logging()
cfg.norm_type = hold
del hold, module, x, output


==========Entering Norm.forward==========
Inputs:
Tensor 'x' shape: torch.Size([32, 512, 128])

==========Entering Norm.LayerNorm==========
Inputs:
Tensor 'x' shape: torch.Size([32, 512, 128])

Outputs:
Tensor 'output' shape: torch.Size([32, 512, 128])
==========Exiting Norm.LayerNorm==========

Outputs:
Tensor 'output' shape: torch.Size([32, 512, 128])
==========Exiting Norm.forward==========


In [6]:
# CosineNorm
hold = cfg.norm_type
cfg.norm_type = 'CosineNorm'
module = Norm(cfg.dim, cfg)
module.enable_logging()

### Optionally disabling printing for sub-functions
#module.disable_function_logging('CosineNorm')
#module.disable_function_logging('LayerNorm')
#module.disable_function_logging('RMSNorm')

x = torch.randn(32,cfg.max_seq_len,cfg.dim)
output = module(x)
module.disable_logging()
cfg.norm_type = hold
del hold, module, x, output


==========Entering Norm.forward==========
Inputs:
Tensor 'x' shape: torch.Size([32, 512, 128])

==========Entering Norm.CosineNorm==========
Inputs:
Tensor 'x' shape: torch.Size([32, 512, 128])

Outputs:
Tensor 'output' shape: torch.Size([32, 512, 128])
==========Exiting Norm.CosineNorm==========

Outputs:
Tensor 'output' shape: torch.Size([32, 512, 128])
==========Exiting Norm.forward==========


# Attention

In [7]:
# first up let's look at training

# Create an instance of multi-head self-attention
module = MQSA(cfg)
print(sum(p.numel() for p in module.parameters())/1e3, 'K parameters')
print(module)

# Initially, logging is disabled by default
module.enable_logging()

### Optionally disabling printing for sub-functions
#module.disable_function_logging('apply_rotary_emb')
#module.disable_function_logging('reshape_for_broadcast')
#module.disable_function_logging('match_headcount')
#module.disable_function_logging('attend')
#module.disable_function_logging('calc_output')

# precompute RoPE frequencies, causal mask, and dummy input data
freqs_cis = precompute_freqs_cis(
    cfg.dim // cfg.num_q_heads,
    cfg.max_seq_len,
    cfg.theta
)
mask = torch.full(
    (cfg.max_seq_len, cfg.max_seq_len),
    float("-inf"),
    device=cfg.device
)
mask = torch.triu(mask, diagonal=1)
x = torch.randn(32,cfg.max_seq_len,cfg.dim)

# Call the forward method - logging will occur
output = module(x, freqs_cis, mask, training=True)

# Disable logging. 
# This isn't actually necessary since we won't be using this object again but that's how you'd do it
module.disable_logging()

# clearing up ram jic we're training later
del module, freqs_cis, mask, x, output

40.96 K parameters
MQSA(
  (Wq): Linear(in_features=128, out_features=128, bias=False)
  (Wk): Linear(in_features=128, out_features=32, bias=False)
  (Wv): Linear(in_features=128, out_features=32, bias=False)
  (Wo): Linear(in_features=128, out_features=128, bias=False)
)

==========Entering MQSA.forward==========
Inputs:
Tensor 'x' shape: torch.Size([32, 512, 128])
Tensor 'freqs_cis' shape: torch.Size([512, 16])
Tensor 'mask' shape: torch.Size([512, 512])
Integer 'cache_len': Value=True

==========Entering MQSA.apply_rotary_emb==========
Inputs:
Tensor 'xq' shape: torch.Size([32, 512, 4, 32])
Tensor 'xk' shape: torch.Size([32, 512, 1, 32])
Tensor 'freqs_cis' shape: torch.Size([512, 16])

==========Entering MQSA.reshape_for_broadcast==========
Inputs:
Tensor 'freqs_cis' shape: torch.Size([512, 16])
Tensor 'x' shape: torch.Size([32, 512, 4, 16])

Outputs:
Tensor 'output' shape: torch.Size([1, 512, 1, 16])
==========Exiting MQSA.reshape_for_broadcast==========

Outputs:
Tensor 'output[0]

In [8]:
# now let's do it for inference

module = MQSA(cfg)
module.enable_logging()
#module.disable_function_logging('apply_rotary_emb')
#module.disable_function_logging('reshape_for_broadcast')
#module.disable_function_logging('match_headcount')
#module.disable_function_logging('attend')
#module.disable_function_logging('calc_output')

# precompute RoPE frequencies, causal mask, and dummy input data
freqs_cis = precompute_freqs_cis(
    cfg.dim // cfg.num_q_heads,
    cfg.max_seq_len,
    cfg.theta
)
mask = torch.full(
    (cfg.max_seq_len, cfg.max_seq_len),
    float("-inf"),
    device=cfg.device
)
mask = torch.triu(mask, diagonal=1)

# setting up for kv caching
cache_len = 420
context_chunk_len = 64
seq_len = cache_len + context_chunk_len
# need to extend the mask with zeros for the cached values
mask = mask[:context_chunk_len, :context_chunk_len]
mask = torch.hstack(
            [torch.zeros((context_chunk_len, cache_len)), mask]
        )

# these don't use seq_len because those entries should already be in the kv cache
freqs_cis = freqs_cis[:context_chunk_len]
x = torch.randn(1,context_chunk_len,cfg.dim)

# Call the forward method - logging will occur
output = module(x, freqs_cis, mask, cache_len)

# Disable logging. 
# This isn't actually necessary since we won't be using this object again but that's how you'd do it
module.disable_logging()

# clearing up ram jic we're training later
del module, freqs_cis, mask, cache_len, context_chunk_len, seq_len, x, output


==========Entering MQSA.forward==========
Inputs:
Tensor 'x' shape: torch.Size([1, 64, 128])
Tensor 'freqs_cis' shape: torch.Size([64, 16])
Tensor 'mask' shape: torch.Size([64, 484])
Integer 'cache_len': Value=420

==========Entering MQSA.apply_rotary_emb==========
Inputs:
Tensor 'xq' shape: torch.Size([1, 64, 4, 32])
Tensor 'xk' shape: torch.Size([1, 64, 1, 32])
Tensor 'freqs_cis' shape: torch.Size([64, 16])

==========Entering MQSA.reshape_for_broadcast==========
Inputs:
Tensor 'freqs_cis' shape: torch.Size([64, 16])
Tensor 'x' shape: torch.Size([1, 64, 4, 16])

Outputs:
Tensor 'output' shape: torch.Size([1, 64, 1, 16])
==========Exiting MQSA.reshape_for_broadcast==========

Outputs:
Tensor 'output[0]' shape: torch.Size([1, 64, 4, 32])
Tensor 'output[1]' shape: torch.Size([1, 64, 1, 32])
==========Exiting MQSA.apply_rotary_emb==========

==========Entering MQSA.match_headcount==========
Inputs:
Tensor 'keys' shape: torch.Size([1, 484, 1, 32])
Tensor 'values' shape: torch.Size([1, 48

# MLP

In [9]:
# GeGLU
module = MLP(
    cfg.dim, 
    cfg.dim * cfg.mlp_hidden_mult, 
    cfg.dim, 
    'GeLU', 
    gated=True, 
    bias=False, 
    dropout_rate = 0.1
)
print(sum(p.numel() for p in module.parameters())/1e3, 'K parameters')
print(module)
module.enable_logging()

x = torch.randn(32,cfg.max_seq_len,cfg.dim)
output = module(x, training=True)
module.disable_logging()
del module, x, output

98.304 K parameters
MLP(
  (Wgate): Linear(in_features=128, out_features=256, bias=False)
  (Wup): Linear(in_features=128, out_features=256, bias=False)
  (Wdown): Linear(in_features=256, out_features=128, bias=False)
  (nonlinearity): GELU(approximate='none')
)

==========Entering MLP.forward==========
Inputs:
Tensor 'x' shape: torch.Size([32, 512, 128])
Integer 'training': Value=True

Outputs:
Tensor 'output' shape: torch.Size([32, 512, 128])
==========Exiting MLP.forward==========


In [10]:
# not gated, testing every other nonlinearity
module = MLP(
    cfg.dim, 
    cfg.dim * cfg.mlp_hidden_mult, 
    cfg.dim, 
    'ReLU', 
    gated=False, 
    bias=False, 
    dropout_rate = 0.1
)
print(sum(p.numel() for p in module.parameters())/1e3, 'K parameters')
print(module)
module.enable_logging()

x = torch.randn(32,cfg.max_seq_len,cfg.dim)
output = module(x, training=True)
module.disable_logging()
del module, x, output

65.536 K parameters
MLP(
  (Wup): Linear(in_features=128, out_features=256, bias=False)
  (Wdown): Linear(in_features=256, out_features=128, bias=False)
  (nonlinearity): ReLU()
)

==========Entering MLP.forward==========
Inputs:
Tensor 'x' shape: torch.Size([32, 512, 128])
Integer 'training': Value=True

Outputs:
Tensor 'output' shape: torch.Size([32, 512, 128])
==========Exiting MLP.forward==========


# ResidualLayer

In [11]:
# TRAINING
module = ResidualLayer(cfg)
print(sum(p.numel() for p in module.parameters())/1e3, 'K parameters')
print(module)

module.enable_logging()
#module.disable_function_logging('attn_connect')
#module.disable_function_logging('mlp_connect')
### enabling printing for sub-modules
#module.pre_attn_norm.enable_logging()
#module.attn.enable_logging()
#module.post_attn_norm.enable_logging()
#module.pre_mlp_norm.enable_logging()
#module.mlp.enable_logging()
#module.post_mlp_norm.enable_logging()

# precompute RoPE frequencies, causal mask, and dummy input data
freqs_cis = precompute_freqs_cis(
    cfg.dim // cfg.num_q_heads,
    cfg.max_seq_len,
    cfg.theta
)
mask = torch.full(
    (cfg.max_seq_len, cfg.max_seq_len),
    float("-inf"),
    device=cfg.device
)
mask = torch.triu(mask, diagonal=1)
x = torch.randn(32,cfg.max_seq_len,cfg.dim)

output = module(x, freqs_cis, mask, training=True)
module.disable_logging()
del module, x, output

139.776 K parameters
ResidualLayer(
  (pre_attn_norm): Norm()
  (attn): MQSA(
    (Wq): Linear(in_features=128, out_features=128, bias=False)
    (Wk): Linear(in_features=128, out_features=32, bias=False)
    (Wv): Linear(in_features=128, out_features=32, bias=False)
    (Wo): Linear(in_features=128, out_features=128, bias=False)
  )
  (pre_mlp_norm): Norm()
  (mlp): MLP(
    (Wgate): Linear(in_features=128, out_features=256, bias=False)
    (Wup): Linear(in_features=128, out_features=256, bias=False)
    (Wdown): Linear(in_features=256, out_features=128, bias=False)
    (nonlinearity): GELU(approximate='none')
  )
)

==========Entering ResidualLayer.forward==========
Inputs:
Tensor 'x' shape: torch.Size([32, 512, 128])
Tensor 'freqs_cis' shape: torch.Size([512, 16])
Tensor 'mask' shape: torch.Size([512, 512])
Integer 'cache_len': Value=True

==========Entering ResidualLayer.attn_connect==========
Inputs:
Tensor 'x' shape: torch.Size([32, 512, 128])
Tensor 'freqs_cis' shape: torch.Size

In [12]:
# INFERENCE
module = ResidualLayer(cfg)
print(sum(p.numel() for p in module.parameters())/1e3, 'K parameters')
print(module)

module.enable_logging()
#module.disable_function_logging('attn_connect')
#module.disable_function_logging('mlp_connect')
#module.pre_attn_norm.enable_logging()
#module.attn.enable_logging()
#module.post_attn_norm.enable_logging()
#module.pre_mlp_norm.enable_logging()
#module.mlp.enable_logging()
#module.post_mlp_norm.enable_logging()

# precompute RoPE frequencies, causal mask, and dummy input data
freqs_cis = precompute_freqs_cis(
    cfg.dim // cfg.num_q_heads,
    cfg.max_seq_len,
    cfg.theta
)
mask = torch.full(
    (cfg.max_seq_len, cfg.max_seq_len),
    float("-inf"),
    device=cfg.device
)
mask = torch.triu(mask, diagonal=1)
# setting up for kv caching
cache_len = 420
context_chunk_len = 64
seq_len = cache_len + context_chunk_len
# need to extend the mask with zeros for the cached values
mask = mask[:context_chunk_len, :context_chunk_len]
mask = torch.hstack(
            [torch.zeros((context_chunk_len, cache_len)), mask]
        )
# these don't use seq_len because those entries should already be in the kv cache
freqs_cis = freqs_cis[:context_chunk_len]
x = torch.randn(1,context_chunk_len,cfg.dim)

output = module(x, freqs_cis, mask, cache_len)
module.disable_logging()
del module, freqs_cis, mask, cache_len, context_chunk_len, seq_len, x, output

139.776 K parameters
ResidualLayer(
  (pre_attn_norm): Norm()
  (attn): MQSA(
    (Wq): Linear(in_features=128, out_features=128, bias=False)
    (Wk): Linear(in_features=128, out_features=32, bias=False)
    (Wv): Linear(in_features=128, out_features=32, bias=False)
    (Wo): Linear(in_features=128, out_features=128, bias=False)
  )
  (pre_mlp_norm): Norm()
  (mlp): MLP(
    (Wgate): Linear(in_features=128, out_features=256, bias=False)
    (Wup): Linear(in_features=128, out_features=256, bias=False)
    (Wdown): Linear(in_features=256, out_features=128, bias=False)
    (nonlinearity): GELU(approximate='none')
  )
)

==========Entering ResidualLayer.forward==========
Inputs:
Tensor 'x' shape: torch.Size([1, 64, 128])
Tensor 'freqs_cis' shape: torch.Size([64, 16])
Tensor 'mask' shape: torch.Size([64, 484])
Integer 'cache_len': Value=420

==========Entering ResidualLayer.attn_connect==========
Inputs:
Tensor 'x' shape: torch.Size([1, 64, 128])
Tensor 'freqs_cis' shape: torch.Size([64, 1

# Full Model

In [14]:
# TRAINING
module = customGPT(cfg)
print(sum(p.numel() for p in module.parameters())/1e3, 'K parameters')
print(module)

module.enable_logging()
### enabling printing for sub-modules
module.layers[0].enable_logging()
module.final_norm.enable_logging()

input_token_ids = torch.randint(tokenizer.vocab_len, (32, cfg.max_seq_len))
target_token_ids = torch.randint(tokenizer.vocab_len, (32, cfg.max_seq_len))

output, loss = module(input_token_ids, target_token_ids=target_token_ids)
print(loss)
module.disable_logging()
del module, input_token_ids, target_token_ids, output

1463.552 K parameters
customGPT(
  (token_embedder): Embedding(512, 128)
  (layers): ModuleList(
    (0-9): 10 x ResidualLayer(
      (pre_attn_norm): Norm()
      (attn): MQSA(
        (Wq): Linear(in_features=128, out_features=128, bias=False)
        (Wk): Linear(in_features=128, out_features=32, bias=False)
        (Wv): Linear(in_features=128, out_features=32, bias=False)
        (Wo): Linear(in_features=128, out_features=128, bias=False)
      )
      (pre_mlp_norm): Norm()
      (mlp): MLP(
        (Wgate): Linear(in_features=128, out_features=256, bias=False)
        (Wup): Linear(in_features=128, out_features=256, bias=False)
        (Wdown): Linear(in_features=256, out_features=128, bias=False)
        (nonlinearity): GELU(approximate='none')
      )
    )
  )
  (final_norm): Norm()
  (criterion): CrossEntropyLoss()
)

==========Entering customGPT.forward==========
Inputs:
Tensor 'input_token_ids' shape: torch.Size([32, 512])
Tensor 'cache_len' shape: torch.Size([32, 512])

=

In [6]:
# Inference
module = customGPT(cfg)
print(sum(p.numel() for p in module.parameters())/1e3, 'K parameters')
print(module)

module.enable_logging()
### enabling printing for sub-modules
#for i in range(cfg.num_layers):
    #module.layers[i].enable_logging()
#module.final_norm.enable_logging()

input_token_ids = torch.randint(tokenizer.vocab_len, (1, cfg.max_seq_len // 3))

output, _ = module(input_token_ids, cache_len = 69)
print(output.shape)
module.disable_logging()
del module, input_token_ids, output

1463.552 K parameters
customGPT(
  (token_embedder): Embedding(512, 128)
  (layers): ModuleList(
    (0-9): 10 x ResidualLayer(
      (pre_attn_norm): Norm()
      (attn): MQSA(
        (Wq): Linear(in_features=128, out_features=128, bias=False)
        (Wk): Linear(in_features=128, out_features=32, bias=False)
        (Wv): Linear(in_features=128, out_features=32, bias=False)
        (Wo): Linear(in_features=128, out_features=128, bias=False)
      )
      (pre_mlp_norm): Norm()
      (mlp): MLP(
        (Wgate): Linear(in_features=128, out_features=256, bias=False)
        (Wup): Linear(in_features=128, out_features=256, bias=False)
        (Wdown): Linear(in_features=256, out_features=128, bias=False)
        (nonlinearity): GELU(approximate='none')
      )
    )
  )
  (final_norm): Norm()
  (criterion): CrossEntropyLoss()
)

==========Entering customGPT.forward==========
Inputs:
Tensor 'input_token_ids' shape: torch.Size([1, 170])
Integer 'cache_len': Value=69

Outputs:
Tensor 'out